<a href="https://colab.research.google.com/github/IAT-ComputationalCreativity-Spring2025/Week11-Genetic-Algorithms/blob/main/DEAP_interactive_images.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Evolving images with human evaluation

## Import the necessary libraries

In [ ]:
! pip install deap numpy matplotlib ipywidgets

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from deap import base, creator, tools, algorithms
import random
from IPython.display import clear_output, display
import ipywidgets as widgets
from google.colab import output

## Define the function to create abstract art

In [ ]:
def create_abstract_art(genome, canvas_size=400):
    """
    Creates abstract art from a genome of parameters.
    
    Each shape is defined by 7 parameters:
    - x, y: position
    - width, height: dimensions
    - r, g, b: color
    """
    # Initialize blank white canvas
    canvas = np.ones((canvas_size, canvas_size, 3))
    
    # Genome structure:
    # Every 7 values represent a shape: [x, y, width, height, r, g, b]
    num_shapes = len(genome) // 7
    
    for i in range(num_shapes):
        # Extract shape parameters
        x = int(genome[i*7] * canvas_size)
        y = int(genome[i*7+1] * canvas_size)
        width = int(max(5, genome[i*7+2] * canvas_size/2))
        height = int(max(5, genome[i*7+3] * canvas_size/2))
        r, g, b = genome[i*7+4], genome[i*7+5], genome[i*7+6]
        
        # Ensure shape is at least partially in canvas
        x = min(max(0, x), canvas_size-1)
        y = min(max(0, y), canvas_size-1)
        
        # Draw rectangle
        x_end = min(x + width, canvas_size)
        y_end = min(y + height, canvas_size)
        
        canvas[y:y_end, x:x_end] = [r, g, b]
    
    return canvas

## Set up the DEAP framework for genetic algorithm

In [ ]:
# Set up the genetic algorithm with DEAP
# If creator was already created, we need to clear it to avoid errors
if 'FitnessMax' in dir(creator):
    del creator.FitnessMax
    del creator.Individual

# Setup for genetic algorithm
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

toolbox = base.Toolbox()

# Define genome: 10 shapes, each with 7 parameters
GENOME_LENGTH = 10 * 7  # 10 shapes with 7 parameters each

# Register gene, individual, and population creation
toolbox.register("attr_float", random.random)
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_float, GENOME_LENGTH)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

# Crossover and mutation
toolbox.register("mate", tools.cxBlend, alpha=0.5)
toolbox.register("mutate", tools.mutGaussian, mu=0, sigma=0.2, indpb=0.2)
toolbox.register("select", tools.selTournament, tournsize=3)

## Create a function to display a grid of artworks with rating sliders

In [ ]:
def display_population(population, generation):
    """
    Displays a grid of artworks from the population with sliders for rating.
    Returns the widgets for later access to their values.
    """
    pop_size = len(population)
    cols = min(4, pop_size)
    rows = (pop_size + cols - 1) // cols  # Ceiling division
    
    fig, axes = plt.subplots(rows, cols, figsize=(cols*4, rows*4))
    if rows == 1 and cols == 1:
        axes = np.array([[axes]])
    elif rows == 1 or cols == 1:
        axes = axes.reshape(rows, cols)
    
    sliders = []
    
    # Flatten axes if needed
    ax_flat = axes.flatten()
    
    for i, ind in enumerate(population):
        if i < len(ax_flat):
            # Generate artwork
            art = create_abstract_art(ind)
            
            # Display
            ax_flat[i].imshow(art)
            ax_flat[i].set_title(f"Artwork {i+1}")
            ax_flat[i].axis('off')
            
            # Create slider for this artwork
            slider = widgets.FloatSlider(
                value=5.0,
                min=0.0,
                max=10.0,
                step=0.5,
                description=f'Rate {i+1}:',
                disabled=False,
                continuous_update=False,
                orientation='horizontal',
                readout=True,
                readout_format='.1f',
            )
            sliders.append(slider)
    
    # Hide any unused axes
    for j in range(len(population), len(ax_flat)):
        ax_flat[j].axis('off')
    
    plt.tight_layout()
    plt.suptitle(f"Generation {generation}", fontsize=16)
    plt.subplots_adjust(top=0.9)
    plt.show()
    
    # Display all sliders
    for slider in sliders:
        display(slider)
    
    return sliders

## Create a function to collect ratings from sliders

In [ ]:
def collect_ratings(population, sliders):
    """
    Collects ratings from sliders and assigns them as fitness values.
    """
    for ind, slider in zip(population, sliders):
        ind.fitness.values = (slider.value,)
    
    return population

## Define the main interactive evolution function

In [ ]:
def run_interactive_evolution(pop_size=8, generations=3):
    """
    Runs the interactive evolution process for the given number of generations.
    """
    # Initialize population
    population = toolbox.population(n=pop_size)
    
    for gen in range(generations):
        print(f"\n--- Generation {gen+1} ---")
        
        # Display artworks and get rating widgets
        sliders = display_population(population, gen+1)
        
        # Wait for user to rate artworks
        rate_button = widgets.Button(description="Submit Ratings")
        display(rate_button)
        
        def on_button_clicked(b):
            b.disabled = True
            b.description = "Processing..."
            # We'll let the code continue after the button is clicked
        
        rate_button.on_click(on_button_clicked)
        
        # This will pause execution until the button is clicked
        # In a real notebook, we would need IPython's run_cell_magic to achieve this
        # For demonstration, we're just showing the structure
        print("After rating the artworks, click the Submit Ratings button.")
        # Waiting for button click...
        
        # Collect ratings and assign fitness
        population = collect_ratings(population, sliders)
        
        # Select parents and create offspring
        offspring = algorithms.varAnd(population, toolbox, cxpb=0.6, mutpb=0.3)
        
        # Replace population with offspring
        population = offspring
    
    # Final evaluation
    print("\n--- Final Generation ---")
    sliders = display_population(population, generations+1)
    
    # Wait for final ratings
    final_button = widgets.Button(description="Submit Final Ratings")
    display(final_button)
    
    def on_final_button_clicked(b):
        b.disabled = True
        b.description = "Processing..."
    
    final_button.on_click(on_final_button_clicked)
    
    print("Rate the final artworks and click Submit Final Ratings.")
    # Waiting for button click...
    
    # Collect final ratings
    population = collect_ratings(population, sliders)
    
    # Display best individual
    best_ind = tools.selBest(population, 1)[0]
    best_art = create_abstract_art(best_ind)
    
    plt.figure(figsize=(8, 8))
    plt.imshow(best_art)
    plt.title(f"Final Best Artwork (Rating: {best_ind.fitness.values[0]:.1f}/10)")
    plt.axis('off')
    plt.show()
    
    return best_ind

## Function to run the evolution non-interactively for testing

In [ ]:
def run_non_interactive_evolution(pop_size=8, generations=3):
    """
    Runs evolution with random fitness values for testing purposes.
    """
    population = toolbox.population(n=pop_size)
    
    for gen in range(generations):
        print(f"Generation {gen+1}")
        
        # Assign random fitness values
        for ind in population:
            ind.fitness.values = (random.uniform(0, 10),)
        
        # Select and create new offspring
        offspring = algorithms.varAnd(population, toolbox, cxpb=0.6, mutpb=0.3)
        
        # Replace population with offspring
        population = offspring
    
    # Final random evaluation
    for ind in population:
        ind.fitness.values = (random.uniform(0, 10),)
    
    # Display the "best" individual
    best_ind = tools.selBest(population, 1)[0]
    best_art = create_abstract_art(best_ind)
    
    plt.figure(figsize=(8, 8))
    plt.imshow(best_art)
    plt.title(f"Best Artwork (Random Rating: {best_ind.fitness.values[0]:.1f}/10)")
    plt.axis('off')
    plt.show()
    
    return best_ind

## Add a Shape-based art function as an alternative

In [ ]:
def create_shape_based_art(genome, canvas_size=400):
    """
    Creates abstract art using various shapes (circles, rectangles, triangles).
    
    Each shape is defined by 9 parameters:
    - shape_type (0-2): rectangle, circle, or triangle
    - x, y: position
    - size, aspect: dimensions
    - r, g, b: color
    - alpha: transparency
    """
    # Initialize blank white canvas
    canvas = np.ones((canvas_size, canvas_size, 3))
    
    # Genome structure:
    # Every 9 values represent a shape: [type, x, y, size, aspect, r, g, b, alpha]
    num_shapes = len(genome) // 9
    
    for i in range(num_shapes):
        # Extract shape parameters
        shape_type = int(genome[i*9] * 3)  # 0: rectangle, 1: circle, 2: triangle
        x = int(genome[i*9+1] * canvas_size)
        y = int(genome[i*9+2] * canvas_size)
        size = int(max(5, genome[i*9+3] * canvas_size/2))
        aspect = max(0.2, genome[i*9+4] * 2)  # 0.2 to 2
        r, g, b = genome[i*9+5], genome[i*9+6], genome[i*9+7]
        alpha = genome[i*9+8]  # Transparency
        
        # Create a shape mask
        shape_mask = np.zeros((canvas_size, canvas_size))
        
        # Draw the appropriate shape
        if shape_type == 0:  # Rectangle
            width = size
            height = int(size * aspect)
            
            x_end = min(x + width, canvas_size)
            y_end = min(y + height, canvas_size)
            
            if x < canvas_size and y < canvas_size:
                shape_mask[y:y_end, x:x_end] = 1
                
        elif shape_type == 1:  # Circle
            radius = size // 2
            y_grid, x_grid = np.ogrid[-y:canvas_size-y, -x:canvas_size-x]
            mask = (x_grid*x_grid + y_grid*y_grid) <= (radius*radius)
            shape_mask[mask] = 1
            
        elif shape_type == 2:  # Triangle
            if x < canvas_size and y < canvas_size:
                # Define three points for triangle
                p1 = (x, y)
                p2 = (x + size, y)
                p3 = (x + size//2, y - int(size * aspect))
                
                # Create a mesh grid
                xx, yy = np.meshgrid(range(canvas_size), range(canvas_size))
                
                # Simple point-in-triangle check using barycentric coordinates
                for i in range(canvas_size):
                    for j in range(canvas_size):
                        if i < canvas_size and j < canvas_size:
                            # Very simple triangle check - can be optimized
                            if (j >= x and j <= x + size and 
                                i <= y and i >= y - int(size * aspect) and
                                (j - x) / size + (i - y) / (-int(size * aspect)) <= 1):
                                shape_mask[i, j] = 1
        
        # Apply the shape to the canvas with transparency
        for c in range(3):
            canvas[:,:,c] = canvas[:,:,c] * (1 - shape_mask * alpha) + shape_mask * alpha * ([r, g, b][c])
    
    return canvas

## Create a function for conducting a human-in-the-loop experiment

In [ ]:
def run_interactive_evolution_colab(pop_size=8, generations=3, art_function=create_abstract_art):
    """
    Runs the interactive evolution process in Google Colab.
    Uses callbacks to handle user input between steps.
    """
    # Initialize population
    population = toolbox.population(n=pop_size)
    
    # Store state that needs to persist across callbacks
    state = {
        'population': population,
        'generation': 0,
        'max_generations': generations,
        'ratings': {},
        'art_function': art_function
    }
    
    def display_generation():
        """Display the current generation and collect ratings"""
        gen = state['generation']
        pop = state['population']
        
        clear_output(wait=True)
        print(f"--- Generation {gen+1} of {state['max_generations']} ---")
        
        # Create a figure with subplots for each individual
        cols = min(4, pop_size)
        rows = (pop_size + cols - 1) // cols  # Ceiling division
        
        fig, axes = plt.subplots(rows, cols, figsize=(cols*4, rows*4))
        if rows == 1 and cols == 1:
            axes = np.array([[axes]])
        elif rows == 1 or cols == 1:
            axes = axes.reshape(rows, cols)
        
        # Flatten axes for easier iteration
        ax_flat = axes.flatten()
        
        # Display each individual
        for i, ind in enumerate(pop):
            if i < len(ax_flat):
                # Generate artwork
                art = state['art_function'](ind)
                
                # Display
                ax_flat[i].imshow(art)
                ax_flat[i].set_title(f"Artwork {i+1}")
                ax_flat[i].axis('off')
        
        # Hide any unused axes
        for j in range(len(pop), len(ax_flat)):
            ax_flat[j].axis('off')
        
        plt.tight_layout()
        plt.suptitle(f"Generation {gen+1}", fontsize=16)
        plt.subplots_adjust(top=0.9)
        plt.show()
        
        # Create rating widgets
        state['ratings'] = {}
        rating_widgets = []
        
        for i in range(len(pop)):
            slider = widgets.FloatSlider(
                value=5.0,
                min=0.0,
                max=10.0,
                step=0.5,
                description=f'Rate {i+1}:',
                disabled=False,
                continuous_update=False,
                orientation='horizontal',
                readout=True,
                readout_format='.1f',
            )
            
            def make_on_change(idx):
                def on_change(change):
                    state['ratings'][idx] = change['new']
                return on_change
            
            slider.observe(make_on_change(i), names='value')
            rating_widgets.append(slider)
            display(slider)
            # Initialize rating
            state['ratings'][i] = 5.0
        
        # Create submit button
        submit_button = widgets.Button(
            description="Submit Ratings",
            button_style='success',
            tooltip='Click to submit your ratings'
        )
        
        def on_submit_clicked(b):
            # Process ratings and move to next generation
            process_generation()
            
        submit_button.on_click(on_submit_clicked)
        display(submit_button)
    
    def process_generation():
        """Process ratings from the current generation and prepare next generation"""
        pop = state['population']
        gen = state['generation']
        
        # Assign fitness values based on ratings
        for i, ind in enumerate(pop):
            ind.fitness.values = (state['ratings'].get(i, 5.0),)
        
        # Check if we're done
        if gen >= state['max_generations'] - 1:
            display_final_result()
            return
        
        # Create next generation
        offspring = algorithms.varAnd(pop, toolbox, cxpb=0.6, mutpb=0.3)
        state['population'] = offspring
        state['generation'] += 1
        
        # Display next generation
        display_generation()
    
    def display_final_result():
        """Display the final best individual"""
        pop = state['population']
        
        # Find the best individual
        best_ind = tools.selBest(pop, 1)[0]
        best_rating = best_ind.fitness.values[0]
        
        clear_output(wait=True)
        print("--- Evolution Complete! ---")
        
        # Display the best artwork
        best_art = state['art_function'](best_ind)
        
        plt.figure(figsize=(8, 8))
        plt.imshow(best_art)
        plt.title(f"Best Artwork (Rating: {best_rating:.1f}/10)")
        plt.axis('off')
        plt.show()
        
        print(f"Final best rating: {best_rating:.1f}/10")
        print("The best individual has the following characteristics:")
        print(f"- Number of shapes: {len(best_ind) // 7}")
        
        # Add option to save this individual
        save_button = widgets.Button(
            description="Save Best Artwork",
            button_style='info',
            tooltip='Click to save the best artwork'
        )
        
        def on_save_clicked(b):
            try:
                # Save the image (in Colab this will download it)
                plt.figure(figsize=(8, 8))
                plt.imshow(best_art)
                plt.axis('off')
                plt.savefig("best_artwork.png", bbox_inches='tight', pad_inches=0)
                plt.close()
                print("Artwork saved as 'best_artwork.png'")
                
                # Also save the genome as a text file
                with open("best_genome.txt", "w") as f:
                    f.write(str(list(best_ind)))
                print("Genome saved as 'best_genome.txt'")
            except Exception as e:
                print(f"Error saving: {e}")
        
        save_button.on_click(on_save_clicked)
        display(save_button)
        
        # Add option to start over
        restart_button = widgets.Button(
            description="Start New Evolution",
            button_style='warning',
            tooltip='Click to start a new evolution process'
        )
        
        def on_restart_clicked(b):
            run_interactive_evolution_colab(pop_size, state['max_generations'], state['art_function'])
        
        restart_button.on_click(on_restart_clicked)
        display(restart_button)
    
    # Start the evolution process
    display_generation()

In [ ]:
def setup_evolution_experiment():
    """
    Provides an interactive UI to configure and start the evolution experiment.
    """
    # Create widgets for configuration
    style_widget = widgets.Dropdown(
        options=[
            ('Rectangle-based', 'rectangles'), 
            ('Shape-based (circles, rectangles, triangles)', 'shapes')
        ],
        value='rectangles',
        description='Art Style:',
        disabled=False,
    )
    
    pop_size_widget = widgets.IntSlider(
        value=8,
        min=4,
        max=16,
        step=2,
        description='Population:',
        disabled=False,
        continuous_update=False
    )
    
    gen_widget = widgets.IntSlider(
        value=3,
        min=2,
        max=6,
        step=1,
        description='Generations:',
        disabled=False,
        continuous_update=False
    )
    
    # Display configuration widgets
    display(widgets.HTML("<h3>Interactive Art Evolution Experiment</h3>"))
    display(widgets.HTML("<p>Configure your experiment parameters:</p>"))
    
    display(style_widget)
    display(pop_size_widget)
    display(gen_widget)
    
    # Start button
    start_button = widgets.Button(
        description="Start Evolution",
        button_style='success',
        tooltip='Click to start the evolution process'
    )
    
    def on_start_clicked(b):
        # Get configuration
        style = style_widget.value
        pop_size = pop_size_widget.value
        generations = gen_widget.value
        
        # Select art function based on style
        art_function = create_abstract_art if style == 'rectangles' else create_shape_based_art
        
        # Update genome length if using shapes
        global GENOME_LENGTH
        if style == 'shapes':
            GENOME_LENGTH = 10 * 9  # 10 shapes with 9 parameters each
            
            # Re-register individual and population creation with new genome length
            toolbox.register("individual", tools.initRepeat, creator.Individual, 
                           toolbox.attr_float, GENOME_LENGTH)
            toolbox.register("population", tools.initRepeat, list, toolbox.individual)
        else:
            GENOME_LENGTH = 10 * 7  # 10 rectangles with 7 parameters each
            
            # Re-register with original genome length
            toolbox.register("individual", tools.initRepeat, creator.Individual, 
                           toolbox.attr_float, GENOME_LENGTH)
            toolbox.register("population", tools.initRepeat, list, toolbox.individual)
        
        # Start the evolution
        run_interactive_evolution_colab(pop_size, generations, art_function)
    
    start_button.on_click(on_start_clicked)
    display(start_button)
    
    # Display instructions
    display(widgets.HTML("""
    <div style="background-color: #f8f9fa; padding: 10px; border-radius: 5px; margin-top: 20px;">
        <h4>Instructions:</h4>
        <p>1. Select your preferred art style, population size, and number of generations.</p>
        <p>2. Click "Start Evolution" to begin the experiment.</p>
        <p>3. For each generation, rate the artworks from 0 to 10 based on your aesthetic preference.</p>
        <p>4. Click "Submit Ratings" to create the next generation based on your preferences.</p>
        <p>5. After the final generation, you'll see the best artwork and have the option to save it.</p>
    </div>
    """))

In [ ]:
def demonstrate_art_styles():
    """
    Shows examples of the different art styles available.
    """
    # Create random genomes
    rectangle_genome = [random.random() for _ in range(10 * 7)]
    shape_genome = [random.random() for _ in range(10 * 9)]
    
    # Create 3 examples of each type
    fig, axes = plt.subplots(2, 3, figsize=(15, 10))
    
    # Rectangle examples
    for i in range(3):
        temp_genome = [random.random() for _ in range(10 * 7)]
        art = create_abstract_art(temp_genome)
        axes[0, i].imshow(art)
        axes[0, i].set_title(f"Rectangle Example {i+1}")
        axes[0, i].axis('off')
    
    # Shape examples
    for i in range(3):
        temp_genome = [random.random() for _ in range(10 * 9)]
        art = create_shape_based_art(temp_genome)
        axes[1, i].imshow(art)
        axes[1, i].set_title(f"Multi-shape Example {i+1}")
        axes[1, i].axis('off')
    
    plt.tight_layout()
    plt.suptitle("Art Style Examples", fontsize=16)
    plt.subplots_adjust(top=0.92)
    plt.show()
    
    # Return to setup
    setup_button = widgets.Button(
        description="Go to Experiment Setup",
        button_style='info',
    )
    
    def on_setup_clicked(b):
        setup_evolution_experiment()
    
    setup_button.on_click(on_setup_clicked)
    display(setup_button)

## Run demonstration

In [ ]:
# Introduction and starting UI
print("""
# Interactive Evolutionary Art Generation

Welcome to this interactive lab on applying genetic algorithms to art generation!

This notebook demonstrates a technique called "Interactive Evolutionary Computation" (IEC), 
where human aesthetic evaluation is used as the fitness function for a genetic algorithm.

## Key Concepts

1. **Representation**: Artworks are encoded as "genomes" - lists of parameters that define shapes.
2. **Evolution**: The population of artworks evolves over generations based on your ratings.
3. **Selection**: Artworks with higher ratings are more likely to "reproduce" and pass their features to the next generation.
4. **Recombination & Mutation**: New artworks are created by combining and slightly altering features from parent artworks.

## How It Works

1. You'll be shown a set of randomly generated artworks.
2. You rate each artwork based on your personal aesthetic preference.
3. The algorithm uses your ratings to select "parents" for the next generation.
4. New "offspring" artworks are created by combining and mutating features from the parents.
5. This process repeats for several generations, with the art evolving based on your preferences.

Click below to see examples of the different art styles, or to start the experiment directly.
""")

# Create buttons for different starting points
example_button = widgets.Button(
    description="See Art Style Examples",
    button_style='info',
    tooltip='View examples of different art styles'
)

start_button = widgets.Button(
    description="Start Experiment Setup",
    button_style='success',
    tooltip='Configure and start the evolution experiment'
)

def on_example_clicked(b):
    demonstrate_art_styles()

def on_start_clicked(b):
    setup_evolution_experiment()

example_button.on_click(on_example_clicked)
start_button.on_click(on_start_clicked)

display(example_button)
display(start_button)